ALBERT:

ALBERT is based on a type of model called a "transformer," which is designed to handle sequences of words and capture their meaning. The special thing about ALBERT is that it's a lighter and more efficient version of another popular model called BERT.

ALBERT achieves its efficiency by using techniques like sharing parameters across layers and reducing the number of parameters it needs to learn. This means that ALBERT can be smaller and faster while still being very good at understanding language.

Despite being smaller, ALBERT can still do many of the same things as bigger models like BERT. It can understand the meaning of words in context, answer questions, summarize text, and much more.

Techniques used in ALBERT:
- Cross-layer parameter sharing: 
  - All Shared: Share paremeters of all the the subluyers.
  - Shared FFN: Only shate the parameter of FFN from first encoder to rest of FFN.
  - Shared Attention: Only share the mulit-headed attention of first encoder to other encoder.
- Factorized Embedding Parameterization: Instead of storing a unique embedding vector for each word in the vocabulary, Factorized Embedding Parameterization factorizes the embedding matrix into two smaller matrices. These matrices are typically called the embedding matrix and the projection matrix.

The embedding matrix contains the embeddings for each word in the vocabulary but with reduced dimensions compared to traditional methods. The projection matrix maps these reduced-dimensional embeddings to the original embedding size.

$v$ vocab size i.e. 30000, $H$ embedding size i.e. 768. The default hiddne layer embedding size would be $V x H$ = 30000 x 768. In order to reduce the hiddne embedding dimension we will use wordpiece embedding of $E$ i.e. 128. We first project vectors to low-dimensional embedding space $VxE$ and then low-dimensional to hiddne space $ExH$.

The process would be first $VxE$ and $ExH$. That is 30000 X 128 and 128 X 768 => 30000 X 768.



Sentence Order Prediction:
Its a binary classification task. For the given pair od sentence, the modle predicts if the order is swapped or not. 

Regular: S1: She cooked momo. S2: it was delicious. Class: Positive (since pair in order)
Negative: S1: it was delicious. S2:She cooked momo. Class: Negative.

In [1]:
from transformers import AlbertTokenizer, AlbertModel
import torch

In [2]:
model = AlbertModel.from_pretrained("albert/albert-base-v2")

config.json: 100%|██████████| 684/684 [00:00<00:00, 59.3kB/s]
model.safetensors: 100%|██████████| 47.4M/47.4M [00:06<00:00, 7.36MB/s]


In [3]:
tokenizer = AlbertTokenizer.from_pretrained("albert/albert-base-v2")

tokenizer_config.json: 100%|██████████| 25.0/25.0 [00:00<00:00, 2.94kB/s]
spiece.model: 100%|██████████| 760k/760k [00:00<00:00, 3.13MB/s]
tokenizer.json: 100%|██████████| 1.31M/1.31M [00:00<00:00, 3.31MB/s]


In [4]:
example = "Kathmandu is city of Temples and Gods."
inputs = tokenizer(example, return_tensors="pt") 
print(inputs)

{'input_ids': tensor([[    2, 28823,    25,   136,    16,  9111,    17,  4769,     9,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [5]:
outtput = model(**inputs)

In [7]:
outtput.last_hidden_state

tensor([[[ 0.1889, -0.0525,  0.1712,  ..., -0.4810, -0.3654,  0.5008],
         [-1.4933, -0.6441,  1.0500,  ..., -0.1704,  1.0213, -0.8828],
         [ 0.4297, -1.0495,  1.1510,  ...,  0.5195,  0.2841, -0.8472],
         ...,
         [-1.4946,  0.5316,  0.4043,  ...,  0.0406,  0.3832, -0.4737],
         [-0.2297, -0.1473,  0.0360,  ..., -0.5322,  1.1365,  0.1591],
         [ 0.0754,  0.1233, -0.0653,  ..., -0.1315,  0.1112,  0.1950]]],
       grad_fn=<NativeLayerNormBackward0>)

In [8]:
print(f'[CLS] agregate repr:\n{outtput.pooler_output}')

[CLS] agregate repr:
tensor([[ 0.7480, -0.7183,  0.7389, -0.3165, -0.4304, -0.9739,  0.6986, -0.6569,
          0.7478, -0.9931,  0.9904,  0.7885, -0.1239, -0.9645, -0.9904, -0.7107,
          0.8002,  0.6817,  0.9925, -0.6690, -0.4718, -0.9980,  0.9947,  0.9788,
          0.8318, -0.6517,  0.7317, -0.9964, -0.9975, -0.6567, -1.0000,  0.8155,
          0.6574,  0.7069,  0.6780, -0.6211,  0.8497,  0.9613, -0.6596,  0.8089,
          0.6778, -0.9946, -0.7629,  0.7876,  0.6490,  0.6558,  0.9810, -0.9790,
          0.5818, -0.6748, -0.6661, -0.6844, -0.6762, -0.9856,  0.2254,  0.8281,
         -0.7373, -0.7002,  1.0000, -0.9769,  0.7069, -0.8502,  0.8438,  0.2927,
         -0.6895,  0.6156,  0.6949,  0.9988, -0.6254,  0.9119,  0.6679,  0.7879,
         -0.3429, -0.8122,  0.9561,  0.7167,  0.3144,  0.6676,  0.7428, -0.9642,
          0.8378,  0.8353, -0.7357,  0.7907, -0.9327, -0.8619,  0.6695, -0.9998,
          0.7482,  0.6929,  0.7075,  0.8176, -0.6775, -1.0000,  0.6586, -0.7398,
       

RoBERT:

"Robustly optimized BERT approach, enhanced version of BERT. 
- Dynamic Masking 
- No NSP task 
- Large batch Size
- Byte-Level BPE (BBPE) as a tokenizer. 

In [51]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaModel, pipeline
import torch

In [13]:
model = RobertaModel.from_pretrained("FacebookAI/roberta-base")

config.json: 100%|██████████| 481/481 [00:00<00:00, 117kB/s]
model.safetensors: 100%|██████████| 499M/499M [01:09<00:00, 7.19MB/s] 
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
model.config

RobertaConfig {
  "_name_or_path": "FacebookAI/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [15]:
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

tokenizer_config.json: 100%|██████████| 25.0/25.0 [00:00<00:00, 4.02kB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 4.48MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 4.36MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 6.68MB/s]


In [44]:
r_example = "You are a beautiful person."

In [45]:
tokenizer.tokenize(r_example)

['You', 'Ġare', 'Ġa', 'Ġbeautiful', 'Ġperson', '.']

The `Ġ` is a white space character in RoBERTa.

In [46]:
inputs = tokenizer(r_example, return_tensors="pt")

In [47]:
output = model(**inputs)

In [48]:
print(f'token repr\nshape:{outtput.last_hidden_state.shape}\n {outtput.last_hidden_state}')

token repr
shape:torch.Size([1, 10, 768])
 tensor([[[ 0.1889, -0.0525,  0.1712,  ..., -0.4810, -0.3654,  0.5008],
         [-1.4933, -0.6441,  1.0500,  ..., -0.1704,  1.0213, -0.8828],
         [ 0.4297, -1.0495,  1.1510,  ...,  0.5195,  0.2841, -0.8472],
         ...,
         [-1.4946,  0.5316,  0.4043,  ...,  0.0406,  0.3832, -0.4737],
         [-0.2297, -0.1473,  0.0360,  ..., -0.5322,  1.1365,  0.1591],
         [ 0.0754,  0.1233, -0.0653,  ..., -0.1315,  0.1112,  0.1950]]],
       grad_fn=<NativeLayerNormBackward0>)


In [49]:
print(f'aggregate repr\nshape:{outtput.pooler_output.shape}\n {outtput.pooler_output}')

aggregate repr
shape:torch.Size([1, 768])
 tensor([[ 0.7480, -0.7183,  0.7389, -0.3165, -0.4304, -0.9739,  0.6986, -0.6569,
          0.7478, -0.9931,  0.9904,  0.7885, -0.1239, -0.9645, -0.9904, -0.7107,
          0.8002,  0.6817,  0.9925, -0.6690, -0.4718, -0.9980,  0.9947,  0.9788,
          0.8318, -0.6517,  0.7317, -0.9964, -0.9975, -0.6567, -1.0000,  0.8155,
          0.6574,  0.7069,  0.6780, -0.6211,  0.8497,  0.9613, -0.6596,  0.8089,
          0.6778, -0.9946, -0.7629,  0.7876,  0.6490,  0.6558,  0.9810, -0.9790,
          0.5818, -0.6748, -0.6661, -0.6844, -0.6762, -0.9856,  0.2254,  0.8281,
         -0.7373, -0.7002,  1.0000, -0.9769,  0.7069, -0.8502,  0.8438,  0.2927,
         -0.6895,  0.6156,  0.6949,  0.9988, -0.6254,  0.9119,  0.6679,  0.7879,
         -0.3429, -0.8122,  0.9561,  0.7167,  0.3144,  0.6676,  0.7428, -0.9642,
          0.8378,  0.8353, -0.7357,  0.7907, -0.9327, -0.8619,  0.6695, -0.9998,
          0.7482,  0.6929,  0.7075,  0.8176, -0.6775, -1.0000,  0.

Lets see masking example

In [52]:
masked_example = "you are a beautiful <mask>."

In [53]:
filler = pipeline("fill-mask", model="FacebookAI/roberta-base")
res = filler(masked_example)
for item in res:
    print(item)

{'score': 0.23962363600730896, 'token': 693, 'token_str': ' woman', 'sequence': 'you are a beautiful woman.'}
{'score': 0.18328218162059784, 'token': 621, 'token_str': ' person', 'sequence': 'you are a beautiful person.'}
{'score': 0.18187165260314941, 'token': 1816, 'token_str': ' girl', 'sequence': 'you are a beautiful girl.'}
{'score': 0.04936479032039642, 'token': 7047, 'token_str': ' soul', 'sequence': 'you are a beautiful soul.'}
{'score': 0.037081778049468994, 'token': 6429, 'token_str': ' lady', 'sequence': 'you are a beautiful lady.'}


ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately)

ELECTRA is a language model that learns by distinguishing between real and fake words  in a sentence, rather than predicting missing words directly. This makes training more efficient while still producing high-quality language representations.

The replaced token is similar to MLM but instead of masking token we replace the token with different token and train the model. The model basically classifies whether the given tokens are actual or replaced. 

How does it work? 

In [55]:
cls_example = ["The", "teacher", "taught", "the", "math", "subject"]
replaced_example = ["a", "teacher", "learnt", "the", "math", "subject"]

We can user BERT classifier for original or replaced category. This is also known as descriminator. First, we Mask the example tokens $T1$= `["The", "teacher", "taught", "the", "math", "subject"]` and mask them to feed into generator $T2$= `["[MASK]", "teacher", "[MASK]", "the", "math", "subject"]`. Now, we feed the $T2$ to descriminator to find of the each token is original or replaced. 

Note: The descriminator is ELECTRA.

One major advantage of ELECTRA over BERT is, BERT uses MLM as a training objective where we mask only 15% of the tokens. Hence, the token signals to the models are aonly 15% of the tokens, since it only predicts those masked tokens. Whereas in ELECTRA, all tokens are training signals since the models classifies whether given tokens are original or replaced. 

In [109]:
from transformers import ElectraTokenizer, ElectraModel
import torch
import torch.nn.functional as F

In [101]:
descriminator = ElectraModel.from_pretrained("google/electra-small-discriminator")

In [102]:
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")

In [110]:
original_text = "The quick brown fox jumps over the lazy dog"
repalced_text = "The quick brown duck jumps over the lazy dog"

In [111]:
inputs = tokenizer(repalced_text, return_tensors="pt")

In [112]:
output = descriminator(**inputs)

In [113]:
logits = output.last_hidden_state
probs = F.softmax(logits, dim=-1)

We can use FFN for classifier to classify whehter the token is original or replaced.

SpanBERT:

Its is mostly used for question answering, where we predict the span of text. Lets see it in action.

In [135]:
span_example = "You are expected to know the laws of your country"
tokens = span_example.split(" ")

In [136]:
tokens

['You',
 'are',
 'expected',
 'to',
 'know',
 'the',
 'laws',
 'of',
 'your',
 'country']

lets mask the tokens. Instead of randomly masking, we will do random contiguous span of tokens.

In [139]:
for i in range(5,9,1):
    tokens[i] = "[MASK]"
tokens

['You',
 'are',
 'expected',
 'to',
 'know',
 '[MASK]',
 '[MASK]',
 '[MASK]',
 '[MASK]',
 'country']

span boundary objective (SBO):

We will train SpanBERT with MLM objective along with SBO. The SBO only uses the toekn present in the span boundary. The boundary includes immediate tokens that are before the start of the span and after the end of span. In our example, `know` and `country` are the span boundary token.

How does the model distinguishes token $X_6$ to $X_7$ since we have same span boundary?

It uses position encoding of the masked token. To predict $X_7$, we use `Know` $R_5$ and `country` $R_10$ as span boundary and position embedding of masked token i.e. $P_2$.


QnA with pretrained SpanBERT

In [140]:
qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/spanbert-finetuned-squadv2",
    tokenizer="mrm8488/spanbert-finetuned-squadv2"
)

config.json: 100%|██████████| 493/493 [00:00<00:00, 329kB/s]
model.safetensors: 100%|██████████| 433M/433M [01:00<00:00, 7.15MB/s] 
Some weights of the model checkpoint at mrm8488/spanbert-finetuned-squadv2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
tokenizer_config.json: 100%|██████████| 24.0/24.0 [00:00<00:00, 11.6kB/s]
vocab.txt: 100%|██████████| 213k/213k [00:00<00:00, 2.94MB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:0

In [141]:
results = qa_pipeline({
'question': "What is machine learning?",
'context': "Machine learning is a subset of artificial intelligence. It is widely for creating a variety of applications such as email filtering and computer vision"
})

In [143]:
results["answer"]

'a subset of artificial intelligence'

Summary:


ALBERT: a lite version of BERT. It uses cross-layer parameter sharing and factorized embedding parameterization techniques to reduce models parameter. It is also used un Sentence Order Prediction (SOP) as a classification task. 

RoBERTa: a variant of BERT which uses dynamic masking techniques only for MLM task. It uses a large batch size for speed and performance. In addition, it uses the BBPE as a tokenizer with vocabulary size of 50K. 

ELECTRA: Instead of MLM task, it detects fake/replaced tokens in sentence. 

SpanBeERT: it uses the MLM objective and SBO objective to predict the masked tokens. It uses the positional embedding to predict tokens just by using immediate boundary tokens.